In [1]:
import sys
sys.path.append("/home/anonymous/ECAP/nyx/")

In [2]:
import jax.numpy as jnp
import numpy as np
import healpy as hp
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.constants import c, h, k_B
from astropy.utils.data import download_file
from sklearn.neighbors import BallTree

from nyx.core.scene import ComponentType
from nyx.core import get_wavelengths, get_healpix_nside
from nyx.core import CatalogQuery, ParameterSpec
from nyx.core.model import EmitterProtocol
from nyx.atmosphere import get_airmass_formula

from nyx.core.spectral import Bandpass
from nyx.core.spectral import SpectralHandler

# Figuring out stellar emitter

In [3]:
import astropy
from astropy.coordinates import SkyCoord, Angle
from nyx.core import Observation

rotation = -0.25*u.deg
location = astropy.coordinates.EarthLocation.from_geodetic(16.5028, -23.27280, 1.8*u.km)
obstime  = astropy.time.Time('2021-06-04T21:14:16', format='isot', scale='utc')
target   = SkyCoord.from_name('sgr A*')
obs = Observation(location, obstime, target, rotation, fov=Angle(3*u.deg))

In [4]:
from nyx.emitter import stars
gaia = stars.GaiaDR3(lim_mag=10)

# Create a flux creator

In [22]:
from nyx.utils.spectra import create_color_grid, PicklesTRDSAtlas1998

In [23]:
%%time
from nyx.units import nixify

G  = Bandpass.from_SVO('GAIA/GAIA3.G')
BP = Bandpass.from_SVO('GAIA/GAIA3.Gbp')
RP = Bandpass.from_SVO('GAIA/GAIA3.Grp')
rp_bp = gaia.catalog['phot_rp_mean_mag'] - gaia.catalog['phot_bp_mean_mag']
spec_grid = create_color_grid(G, (RP, BP), [np.nanmin(rp_bp), np.nanmax(rp_bp)], PicklesTRDSAtlas1998())

wvl_nixed = nixify(spec_grid.wvl, 'wavelength')
med_nixed = nixify(np.nanmedian(spec_grid.flx, axis=-1)*u.ph, 'flux', wavelength=spec_grid.wvl)

CPU times: user 3.11 s, sys: 3.46 s, total: 6.57 s
Wall time: 6.6 s


In [20]:
%%time
wvl = get_wavelengths()
new_samp = SpectralHandler.resample(wvl_nixed, med_nixed, wvl)
interpol = RegularGridInterpolator((spec_grid.points), new_samp, method='linear', bounds_error=False)
new_spec = interpol(rp_bp[::100])

CPU times: user 40.2 ms, sys: 56.9 ms, total: 97.1 ms
Wall time: 30.8 ms


In [25]:
np.isfinite(rp_bp).sum()/len(rp_bp)

np.float64(0.9993901756045351)

In [26]:
gaia.catalog

array([(      7632157690368,  45.03434283,  0.2353896 ,  45.46600983,  -6.8343431 , 8.068802,  8.554075, 7.4199953),
       (     16733192740608,  45.15296534,  0.38634212,  23.34281993,  -2.41978702, 9.925234, 10.204026, 9.4804735),
       (     30343944744320,  45.09499151,  0.47683613,  19.3533002 ,   4.01393759, 9.899   ,  9.873377, 9.918395 ),
       ...,
       (6917518002460972672, 314.7406513 , -0.22811331,  11.3763878 ,   0.99993277, 9.12541 ,  9.7821  , 8.348033 ),
       (6917520888679058304, 315.04060953, -0.2368594 , -12.99654254, -20.89523157, 9.437743,  9.979468, 8.742322 ),
       (6917521541514092416, 314.96073173, -0.22130001,   4.63956865,   2.59169662, 9.743196,  9.874115, 9.514187 )],
      shape=(482106,), dtype=[('source_id', '<i8'), ('ra', '<f8'), ('dec', '<f8'), ('pmra', '<f8'), ('pmdec', '<f8'), ('phot_g_mean_mag', '<f8'), ('phot_bp_mean_mag', '<f8'), ('phot_rp_mean_mag', '<f8')])